In [1]:
# pip install "gymnasium[box2d]" pyglet

In [2]:
import gymnasium as gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy
import os
import torch 

device = torch.device("mps" if torch.mps.is_available() else "cpu")

In [3]:
enviorment_name = "CarRacing-v3"
env = gym.make(enviorment_name, render_mode="human")

In [4]:
env.reset()

2024-12-20 17:16:49.627 Python[12980:442175] +[IMKClient subclass]: chose IMKClient_Modern
2024-12-20 17:16:49.627 Python[12980:442175] +[IMKInputSession subclass]: chose IMKInputSession_Modern


(array([[[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        ...,
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]]], dtype=uint8),
 {})

In [5]:
# env.action_space

In [6]:
env.observation_space

Box(0, 255, (96, 96, 3), uint8)

In [7]:
env.reset()
env.render() 

In [8]:
# episodes = 5
# for episode in range(1, episodes + 1):
#     state = env.reset()
#     done = False
#     score = 0
# 
#     while not done:
#         action = env.action_space.sample()
#         n_state, reward, terminated, truncated, info = env.step(action)
#         done = terminated or truncated
#         score += reward
# 
#     print(f'Episode:{episode}, Score:{score}')
# env.close() 

In [9]:
env = gym.make(enviorment_name)
env = DummyVecEnv([lambda: env])

In [10]:
log_path = os.path.join('Training', 'Logs')
model = PPO("CnnPolicy", env, verbose=1, tensorboard_log=log_path, device=device)


Using mps device
Wrapping the env in a VecTransposeImage.


In [11]:
# model.learn(total_timesteps=100000)

In [12]:
PPO_path = os.path.join('Training', 'Saved Models', 'PPO_model_CarRacing')
model.save(PPO_path)

In [13]:
del model

In [14]:
model = PPO.load(PPO_path, env=env)

Wrapping the env in a VecTransposeImage.


In [15]:
from stable_baselines3.common.evaluation import evaluate_policy

env = gym.make("CarRacing-v3", render_mode="human")
PPO_path = os.path.join("Training", "Saved Models", "PPO_model_CarRacing")
model = PPO.load(PPO_path, env=env)

def evaluate_policy_with_rendering(model, env, n_eval_episodes=10, render=True):
    """
    Evaluate the policy of a loaded model with optional rendering.
    """
    episode_rewards = []

    for episode in range(n_eval_episodes):
        reset_output = env.reset()
        state = reset_output[0] if isinstance(reset_output, tuple) else reset_output
        done = False
        total_reward = 0

        while not done:
            if render:
                env.render()  # Render each frame

            # Get the action from the model
            action, _states = model.predict(state, deterministic=True)
            
            # Adjust for environments returning 4 or 5 values
            step_output = env.step(action)
            if len(step_output) == 5:
                state, reward, terminated, truncated, info = step_output
            else:
                state, reward, terminated, truncated = step_output

            # Convert reward to scalar to avoid warnings
            total_reward += reward.item() if hasattr(reward, 'item') else float(reward)

            # Combine termination flags
            done = terminated or truncated

        episode_rewards.append(total_reward)
        print(f"Episode {episode + 1}: Total Reward = {total_reward}")

    # Calculate mean and standard deviation of rewards
    mean_reward = sum(episode_rewards) / n_eval_episodes
    std_reward = (sum([(x - mean_reward) ** 2 for x in episode_rewards]) / n_eval_episodes) ** 0.5

    return mean_reward, std_reward

# Step 5: Evaluate the loaded model
mean_reward, std_reward = evaluate_policy_with_rendering(model, env, n_eval_episodes=10, render=True)
print(f"Mean reward: {mean_reward}, Std reward: {std_reward}")

# Close the environment
env.close()

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.
Episode 1: Total Reward = -29.230769230769347
Episode 2: Total Reward = -20.000000000000078
Episode 3: Total Reward = -8.04597701149466


KeyboardInterrupt: 